In [36]:
import numpy as np

N = 8
lattice_basis = 2 * N * np.eye(N).astype(int)

print(lattice_basis)

[[16  0  0  0  0  0  0  0]
 [ 0 16  0  0  0  0  0  0]
 [ 0  0 16  0  0  0  0  0]
 [ 0  0  0 16  0  0  0  0]
 [ 0  0  0  0 16  0  0  0]
 [ 0  0  0  0  0 16  0  0]
 [ 0  0  0  0  0  0 16  0]
 [ 0  0  0  0  0  0  0 16]]


In [37]:
list_bn = [
827733596782305800412928022203,
949647717115293590693294029007,
513755983108074989775373368979,
677679034424075998463761252321,
453168678084622963159703254177,
832782910923539364543910431379,
252562957778396321268184912171,
651496223062243390455276796757,
432919332491435708844136579631,
223770585052094118295318801799,
532148703589048376478494845573,
243856773163704399644951505359,
164819263334667219851593378483,
401469735249463137163038448637,
426751380869555974285167310687,
560854616065731349966291565407,
683800733848288692548599026941,
720403676000154697037384804609,
107301258962892384849699294203,
520071830033615222504245399291,
994869210101506594790719948613,
245219053806416037970010902657,
686041058007528223688600531911,
627290270131015070770970266369,
986406199575879090372513368911,
161242727312477951737327198843,
774858181281373313211111520339,
684559254088515873739510084459,
257694642268698126951808240063,
571233875063041256502141730223,
644801812715761815209851630957,
959697197099941066839436226867,
624429478423882721953553862137,
174367152486141929170585530673,
599558656796597046233579141381,
433361790406333033396425865133,
327914293338207349929854266927,
469829830404491409659825996297,
440912965020782384634858339911,
117716796822761004609505655351,
202743088259468212450682455949,
825678387625860526264958140667,
186563286821803413392902752181,
344598372208490315697621082241,
250491666241092516038026895957,
455905910983924930564630078541,
764508361786508023846708165651,
368402894141896908753430052087,
632529273538568546719862413771,
238262282320450157628785827507,
527663446629057224413918255997,
733182256888608076021693398743,
294525687336161906252161965089,
168418874893897148434218110009,
967500129217854143886182418299,
918282852294568912813037878951,
351474820193810768760797418683,
472054784060326913010893152441,
213047323530044816411212536661,
840631896797768973136803074339,
994618666924895047842532997701,
696690223952403242478248080393,
302823001073534810462474580767,
415854418638180287264172792169]

# slice the list to N portions
sliced_list_bn = [list_bn[i::N] for i in range(N)]

print(len(sliced_list_bn))

8


In [38]:
list_basis = []
for i in range(N):
    for j in range(N):
        list_basis.append(sliced_list_bn[i][j] * lattice_basis[i][j])


modulo = 2147483647 # Mersenne prime 2^31 - 1
print(modulo)


for i in range(len(list_basis)):
    list_basis[i] = list_basis[i] % modulo

sliced_list_basis = [list_basis[i::N] for i in range(N)]
print(len(sliced_list_basis))


2147483647
8


In [39]:
def matrix_eye(N):
    result = [[int(0) for _ in range(N)] for _ in range(N)]
    for i in range(N):
        result[i][i] = int(1)
    return result

In [40]:
def matrix_if_identical(matrix_a, matrix_b):
    for i in range(N):
        for j in range(N):
            if matrix_a[i][j] != matrix_b[i][j]:
                return False
    return True


In [41]:
secret = [[[np.random.randint(-10, 10) for _ in range(N)] for _ in range(N)] for _ in range(N)]
shared_secret = [[] for _ in range(N)]
public = [[] for _ in range(N)]
public_cache = [[] for _ in range(N)]
public_rep = [[] for _ in range(N)]
public_cache_rep = [[] for _ in range(N)]

print(len(secret))
print(public_rep)
print(public_cache_rep)

8
[[], [], [], [], [], [], [], []]
[[], [], [], [], [], [], [], []]


In [42]:
def matrix_mul(matrix_a, matrix_b, modulo):
    result = [[int(0) for _ in range(N)] for _ in range(N)]
    for i in range(N):
        for j in range(N):
            for k in range(N):
                result[i][j] += matrix_a[i][k] * matrix_b[k][j]
                result[i][j] %= modulo
    return result

In [43]:
def matrix_add(matrix_a, matrix_b, modulo):
    result = [[int(0) for _ in range(N)] for _ in range(N)]
    for i in range(N):
        for j in range(N):
            result[i][j] = matrix_a[i][j] + matrix_b[i][j]
            result[i][j] %= modulo
    return result

In [44]:
def matrix_mul_and_add(matrix_a, matrix_b, matrix_c, modulo):
    result = [[int(0) for _ in range(N)] for _ in range(N)]
    for i in range(N):
        for j in range(N):
            for k in range(N):
                result[i][j] += matrix_a[i][k] * matrix_b[k][j]
                result[i][j] %= modulo
            result[i][j] += matrix_c[i][j]
            result[i][j] %= modulo
    return result

In [45]:
def matrix_div(matrix, divisor=100):
    result = [[int(0) for _ in range(N)] for _ in range(N)]
    for i in range(N):
        for j in range(N):
            result[i][j] = matrix[i][j] // divisor
    return result

In [46]:
def print_matrix(matrix):
    for i in range(N):
        for j in range(N):
            print(f"{int(matrix[i][j]):15}", end=' ')
        print()
    print()

In [47]:
a = matrix_mul_and_add(secret[0], sliced_list_basis, np.random.randint(-1, 1, (N, N)), modulo)
a = matrix_mul(a, secret[1], modulo)
a = matrix_div(a, 10000000)

e = matrix_mul_and_add(sliced_list_basis, secret[1], np.random.randint(-1, 1, (N, N)), modulo)
e = matrix_mul(secret[0], e, modulo)
e = matrix_div(e, 10000000)

print(matrix_if_identical(a, e))

for i in range(N):
    for j in range(N):
        print(f"{int(a[i][j]):15}", end=" ")
    print()

print()

for i in range(N):
    for j in range(N):
        print(f"{int(e[i][j]):15}", end=" ")
    print()

True
            183              17             127              66              91             204              57             177 
            208              38              58             111              66             157              27              97 
            151              63             127              75              88             158             112             109 
             71             177             155             154             202              85              14             177 
             39             181              26              65             150             153              98             209 
            174              68              24              74             108             176              11              61 
            155              10               9              76              11             103              95              34 
             88             208             115             114             188             

C:\Users\Admin\AppData\Local\Temp\ipykernel_15948\3855214896.py:6: RuntimeWarning: overflow encountered in scalar multiply
  result[i][j] += matrix_a[i][k] * matrix_b[k][j]
C:\Users\Admin\AppData\Local\Temp\ipykernel_15948\3855214896.py:6: RuntimeWarning: overflow encountered in scalar add
  result[i][j] += matrix_a[i][k] * matrix_b[k][j]


In [48]:
def secret_renew(secret_matrix, public_matrix, id: int, left_or_right: int, modulo):
    if left_or_right == 0:
        a = matrix_mul(secret_matrix, public_matrix, modulo)
        a = matrix_div(a, 10000000)
        return a
    else:
        a = matrix_mul(public_matrix, secret_matrix, modulo)
        a = matrix_div(a, 10000000)
        return a


In [49]:
def public_renew(secret_matrix, id: int, left_or_right: int, modulo):
    if left_or_right == 0:
        a = matrix_mul_and_add(secret_matrix, sliced_list_basis, np.random.randint(-1, 1, (N, N)), modulo)
        return a
    else:
        a = matrix_mul_and_add(sliced_list_basis, secret_matrix, np.random.randint(-1, 1, (N, N)), modulo)
        return a


In [50]:
print_matrix(secret[0])


              6               2              -7               0             -10             -10               1              -6 
              0               8              -4             -10               3               0               1               0 
             -6              -9              -5               7              -9              -8              -4              -5 
             -1              -9               9               4              -5               5              -7               1 
              6               6              -5              -5               9               1               9              -3 
              2              -4               6               0               8              -9               6              -5 
             -5              -1             -10              -9               6              -4              -5              -9 
              3              -8              -7               4               7               9  

In [51]:
def multi_DH(n, round=0, verbose=False):
    '''
    This function computes the multi-party Diffie-Hellman key exchange protocol.
    '''
    if verbose:
        print(f"Round {round}:")
    match round:
        case 0:
            public[0] = public_renew(secret[0], 0, 0, modulo)
            public[1] = public_renew(secret[1], 1, 1, modulo)
            public[2] = public_renew(secret[2], 2, 0, modulo)
            public[3] = public_renew(secret[3], 3, 1, modulo)
            public[4] = public_renew(secret[4], 4, 0, modulo)
            public[5] = public_renew(secret[5], 5, 1, modulo)
            public[6] = public_renew(secret[6], 6, 0, modulo)
            public[7] = public_renew(secret[7], 7, 1, modulo)   
            public_rep[0] = [0]
            public_rep[1] = [1]
            public_rep[2] = [2]
            public_rep[3] = [3]
            public_rep[4] = [4]
            public_rep[5] = [5]
            public_rep[6] = [6]
            public_rep[7] = [7]
        case 1:
            secret[0] = secret_renew(secret[0], public[1], 0, 0, modulo)
            secret[1] = secret_renew(secret[1], public[0], 1, 1, modulo)
            secret[2] = secret_renew(secret[2], public[3], 2, 0, modulo)
            secret[3] = secret_renew(secret[3], public[2], 3, 1, modulo)
            secret[4] = secret_renew(secret[4], public[5], 4, 0, modulo)
            secret[5] = secret_renew(secret[5], public[4], 5, 1, modulo)
            secret[6] = secret_renew(secret[6], public[7], 6, 0, modulo)
            secret[7] = secret_renew(secret[7], public[6], 7, 1, modulo)
            public[0] = public_renew(secret[0], 0, 0, modulo)
            public[1] = public_renew(secret[1], 1, 0, modulo)
            public[2] = public_renew(secret[2], 2, 1, modulo)
            public[3] = public_renew(secret[3], 3, 1, modulo)
            public[4] = public_renew(secret[4], 4, 0, modulo)
            public[5] = public_renew(secret[5], 5, 0, modulo)
            public[6] = public_renew(secret[6], 6, 1, modulo)
            public[7] = public_renew(secret[7], 7, 1, modulo)
            public_rep[0] = [0, 1]
            public_rep[1] = [0, 1]
            public_rep[2] = [2, 3]
            public_rep[3] = [2, 3]
            public_rep[4] = [4, 5]
            public_rep[5] = [4, 5]
            public_rep[6] = [6, 7]
            public_rep[7] = [6, 7]
        case 2:
            secret[0] = secret_renew(secret[0], public[3], 0, 0, modulo)
            secret[1] = secret_renew(secret[1], public[2], 1, 0, modulo)
            secret[2] = secret_renew(secret[2], public[1], 2, 1, modulo)
            secret[3] = secret_renew(secret[3], public[0], 3, 1, modulo)
            secret[4] = secret_renew(secret[4], public[7], 4, 0, modulo)
            secret[5] = secret_renew(secret[5], public[6], 5, 0, modulo)
            secret[6] = secret_renew(secret[6], public[5], 6, 1, modulo)
            secret[7] = secret_renew(secret[7], public[4], 7, 1, modulo)
            public[0] = public_renew(secret[0], 0, 0, modulo)
            public[1] = public_renew(secret[1], 1, 0, modulo)
            public[2] = public_renew(secret[2], 2, 0, modulo)
            public[3] = public_renew(secret[3], 3, 0, modulo)
            public[4] = public_renew(secret[4], 4, 1, modulo)
            public[5] = public_renew(secret[5], 5, 1, modulo)
            public[6] = public_renew(secret[6], 6, 1, modulo)
            public[7] = public_renew(secret[7], 7, 1, modulo)
            public_rep[0] = [0, 1, 2, 3]
            public_rep[1] = [0, 1, 2, 3]
            public_rep[2] = [0, 1, 2, 3]
            public_rep[3] = [0, 1, 2, 3]
            public_rep[4] = [4, 5, 6, 7]
            public_rep[5] = [4, 5, 6, 7]
            public_rep[6] = [4, 5, 6, 7]
            public_rep[7] = [4, 5, 6, 7]
        case 3:
            secret[0] = secret_renew(secret[0], public[7], 0, 0, modulo)
            secret[1] = secret_renew(secret[1], public[6], 1, 0, modulo)
            secret[2] = secret_renew(secret[2], public[5], 2, 0, modulo)
            secret[3] = secret_renew(secret[3], public[4], 3, 0, modulo)
            secret[4] = secret_renew(secret[4], public[3], 4, 1, modulo)
            secret[5] = secret_renew(secret[5], public[2], 5, 1, modulo)
            secret[6] = secret_renew(secret[6], public[1], 6, 1, modulo)
            secret[7] = secret_renew(secret[7], public[0], 7, 1, modulo)
            public_rep[0] = [0, 1, 2, 3, 4, 5, 6, 7]
            public_rep[1] = [0, 1, 2, 3, 4, 5, 6, 7]
            public_rep[2] = [0, 1, 2, 3, 4, 5, 6, 7]
            public_rep[3] = [0, 1, 2, 3, 4, 5, 6, 7]
            
        

In [52]:
# Run the multi-party Diffie-Hellman key exchange protocol
for i in range(4):
    multi_DH(N, round=i, verbose=True)
    for j in range(N):
        print(f"Secret {j}:")
        print(secret[j][0])

for i in range(N):
    print(matrix_if_identical(secret[i], secret[0]))


Round 0:
Secret 0:
[6, 2, -7, 0, -10, -10, 1, -6]
Secret 1:
[8, 0, -2, 0, -7, -5, 7, -4]
Secret 2:
[-3, 4, -4, -3, 4, -5, -7, -4]
Secret 3:
[3, -7, 6, -1, -3, 0, 7, -1]
Secret 4:
[9, -10, 3, -9, -6, -7, -5, -4]
Secret 5:
[0, 1, -5, 1, -4, 2, 6, 9]
Secret 6:
[-6, -9, 3, 7, 6, 3, -5, 8]
Secret 7:
[5, 7, -5, -8, 2, -9, -9, -4]
Round 1:
Secret 0:
[183, 17, 127, 66, 91, 204, 57, 177]
Secret 1:
[183, 17, 127, 66, 91, 204, 57, 177]
Secret 2:
[35, 109, 94, 94, 181, 91, 211, 32]
Secret 3:
[35, 109, 94, 94, 181, 91, 211, 32]
Secret 4:
[144, 175, 73, 152, 89, 50, 187, 60]
Secret 5:
[144, 175, 73, 152, 89, 50, 187, 60]
Secret 6:
[46, 96, 7, 60, 185, 165, 194, 43]
Secret 7:
[46, 96, 7, 60, 185, 165, 194, 43]
Round 2:
Secret 0:
[179, 97, 206, 35, 191, 39, 208, 172]
Secret 1:
[179, 97, 206, 35, 191, 39, 208, 172]
Secret 2:
[179, 97, 206, 35, 191, 39, 208, 172]
Secret 3:
[179, 97, 206, 35, 191, 39, 208, 172]
Secret 4:
[68, 121, 93, 178, 11, 175, 137, 37]
Secret 5:
[68, 121, 93, 178, 11, 175, 137, 37]


C:\Users\Admin\AppData\Local\Temp\ipykernel_15948\3855214896.py:6: RuntimeWarning: overflow encountered in scalar multiply
  result[i][j] += matrix_a[i][k] * matrix_b[k][j]
C:\Users\Admin\AppData\Local\Temp\ipykernel_15948\3855214896.py:6: RuntimeWarning: overflow encountered in scalar add
  result[i][j] += matrix_a[i][k] * matrix_b[k][j]
